tb - Train coefficients of a simple analytic model for SW radiative transfer via stochastic gradient descent

Then calculate (1) Number of free parameters; and (2) error statistics of the net shortwave flux prediction (in $W m^{-2}$)

# Imports

In [19]:
from ml4rt.io.example_io import *
from ml4rt.utils.normalization import *
from ml4rt.machine_learning.neural_net import *
#from ml4rt.machine_learning.u_net_architecture import *
#from ml4rt.machine_learning.u_net_pp_architecture import *

import numpy as np

# Read File

In [2]:
read_file

<function ml4rt.io.example_io.read_file(netcdf_file_name, exclude_summit_greenland=False, max_heating_rate_k_day=41.5, id_strings_to_read=None, allow_missing_ids=False)>

In [3]:
path_data = '/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/2021_Ryan_Lagerquist_SW/'

In [4]:
read2020 = read_file(path_data+'tropical_sites/learning_examples_2020.nc',
                     exclude_summit_greenland=True,
                     max_heating_rate_k_day=np.inf)

/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/ml4rt/notebooks_tbeucler/ml4rt/io/example_io.py:304: UserWarning: Removing 0 of 264638 examples (profiles), because they are at Summit GL.
  warnings.warn(warning_string)


In [5]:
read2020.keys()

dict_keys(['example_id_strings', 'scalar_predictor_names', 'vector_predictor_names', 'scalar_target_names', 'vector_target_names', 'scalar_predictor_matrix', 'vector_predictor_matrix', 'scalar_target_matrix', 'vector_target_matrix', 'valid_times_unix_sec', 'standard_atmo_flags', 'heights_m_agl'])

In [6]:
read2020['scalar_predictor_matrix'].shape

(264638, 6)

In [7]:
read2020['scalar_predictor_names']

['zenith_angle_radians',
 'albedo',
 'latitude_deg_n',
 'longitude_deg_e',
 'column_liquid_water_path_kg_m02',
 'column_ice_water_path_kg_m02']

In [8]:
read2020['vector_predictor_matrix'].shape

(264638, 73, 12)

In [9]:
read2020['vector_predictor_names']

['pressure_pascals',
 'temperature_kelvins',
 'specific_humidity_kg_kg01',
 'liquid_water_content_kg_m03',
 'ice_water_content_kg_m03',
 'liquid_water_path_kg_m02',
 'ice_water_path_kg_m02',
 'vapour_path_kg_m02',
 'upward_liquid_water_path_kg_m02',
 'upward_ice_water_path_kg_m02',
 'upward_vapour_path_kg_m02',
 'relative_humidity_unitless']

In [10]:
read2020['scalar_target_names']

['shortwave_surface_down_flux_w_m02', 'shortwave_toa_up_flux_w_m02']

In [11]:
read2020['scalar_target_matrix'].shape

(264638, 2)

In [12]:
read2020['vector_target_names']

['shortwave_down_flux_w_m02',
 'shortwave_up_flux_w_m02',
 'shortwave_heating_rate_k_day01',
 'shortwave_down_flux_increment_w_m03',
 'shortwave_up_flux_increment_w_m03']

In [13]:
read2020['vector_target_matrix'].shape

(264638, 73, 5)

# Data generator

## Load normalization parameters from non tropical sites

In [14]:
path_norm_param = path_data+'non_tropical_sites/learning_examples_20170101-20181224.nc'

In [15]:
norm_param = read_file(path_norm_param,
                     exclude_summit_greenland=True,
                     max_heating_rate_k_day=np.inf)

/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/ml4rt/notebooks_tbeucler/ml4rt/io/example_io.py:304: UserWarning: Removing 0 of 100000 examples (profiles), because they are at Summit GL.
  warnings.warn(warning_string)


In [16]:
norm_param.keys()

dict_keys(['example_id_strings', 'scalar_predictor_names', 'vector_predictor_names', 'scalar_target_names', 'vector_target_names', 'scalar_predictor_matrix', 'vector_predictor_matrix', 'scalar_target_matrix', 'vector_target_matrix', 'valid_times_unix_sec', 'standard_atmo_flags', 'heights_m_agl'])

## Apply normalization 
Following [https://github.com/thunderhoser/ml4rt/blob/8ad36b52ead1c4870bdccaa804dee9a52144cce0/ml4rt/machine_learning/neural_net.py#L213]

In [17]:
normalize_data

<function ml4rt.utils.normalization.normalize_data(new_example_dict, training_example_dict, normalization_type_string, min_normalized_value=-1.0, max_normalized_value=1.0, separate_heights=False, apply_to_predictors=True, apply_to_vector_targets=True, apply_to_scalar_targets=True)>

The normalization is automatically applied as part of data_generator (see _read_file_for_generator)

In [18]:
data_generator

<function ml4rt.machine_learning.neural_net.data_generator(option_dict, for_inference, net_type_string)>

## Build data generator that takes 